In [5]:
import requests
import datetime
import feedparser
from textblob import TextBlob

In [3]:
# Replace with your Finnhub API Key
FINNHUB_API_KEY = "cv1bm91r01qhkk81os0gcv1bm91r01qhkk81os10"

In [28]:
def get_finnhub_news(stock_ticker, start_date, end_date):
    """
    Fetches stock news from Finnhub before the given date.
    """
    url = "https://finnhub.io/api/v1/company-news"
    params = {
        "symbol": stock_ticker.upper(),
        "from": start_date,
        "to": end_date,
        "token": FINNHUB_API_KEY
    }

    response = requests.get(url, params=params)
    data = response.json()

    if not isinstance(data, list):
        print(f"Error fetching news: {data}")
        return []

    articles = [article["headline"] + " " + (article.get("summary") or "") for article in data]
    return articles

In [42]:
def analyze_sentiment(articles):
    """
    Performs sentiment analysis on the articles and returns an overall sentiment score.
    """
    sentiment_scores = [TextBlob(article).sentiment.polarity for article in articles]

    if not sentiment_scores:
        return "No articles found"
    
    count = len(articles)
    for i in range(0, count):
        sentiment_scores[i] = sentiment_scores[i] * ((count - i) / count)

    avg_sentiment = (sum(sentiment_scores) / len(sentiment_scores))*10

    return avg_sentiment

    # if avg_sentiment > 0.5:
    #     return "Positive"
    # elif avg_sentiment < -0.5:
    #     return "Negative"
    # else:
    #     return "Neutral"

In [43]:
if __name__ == "__main__":
    stock_ticker = input("Enter stock ticker (e.g., AAPL, TSLA): ")
    end_date = input("Enter date (YYYY-MM-DD): ")

    try:
        end_date_object = datetime.datetime.strptime(end_date, "%Y-%m-%d")  # Validate date format
    except ValueError:
        print("Invalid date format. Use YYYY-MM-DD.")
        exit()
    
    start_date_object = end_date_object + datetime.timedelta(-30)
    start_date = start_date_object.strftime("%Y-%m-%d")
    end_date = end_date_object.strftime("%Y-%m-%d")

    print(f"Fetching latest news articles about {stock_ticker} from Finnhub before {end_date}...")

    articles = get_finnhub_news(stock_ticker, start_date, end_date)

    if not articles:
        print("No articles found.")
    else:
        sentiment_result = analyze_sentiment(articles)
        print(f"Overall Sentiment: {sentiment_result}")

Fetching latest news articles about AVGO from Finnhub before 2025-03-01...
Overall Sentiment: 0.8558830748816919
